# Introduction

In this notebook I create a way of using the aggregated model in the explaianability dashboard against targets. Basically for each day trhough the year it creates different aggregation for the day agains the previous ones, and then, it creates a prediction with the targets model.

### Instalations and imports

In [2]:
!pip install plotly s3fs darts shap lightgbm minepy dcor

In [3]:
# General
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import os
import numpy as np
# import xlsxwriter
import datetime
import boto3
import s3fs

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV

# Plots
import matplotlib.pyplot as plt
import seaborn as sns

#Warnings
import warnings
warnings.filterwarnings("ignore")


import darts
from darts import TimeSeries
from darts.utils.timeseries_generation import (
    gaussian_timeseries,
    linear_timeseries,
    sine_timeseries,
)

from darts.metrics import mape, smape, mae
from darts.dataprocessing.transformers import Scaler
from darts.utils.timeseries_generation import datetime_attribute_timeseries

from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import RandomForestRegressor

import lightgbm

from darts.models import LightGBMModel

from darts.models import LightGBMModel, RandomForest, LinearRegressionModel
from darts.utils.statistics import check_seasonality, plot_acf, plot_residuals_analysis

from darts.explainability.shap_explainer import ShapExplainer
import pickle
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from darts.models import LinearRegressionModel, LightGBMModel, RandomForest
from calendar import month_name as mn
import os

import shap


# 1. Aggregation logic

In [4]:
# Use own bucket and prefix
S3_BUCKET_NPS = 'iberia-data-lake' # In this case: iberia-data-lake
S3_BUCKET_NPS_PREFIX = 'customer/nps_explainability_model' # In this case: sagemaker/sagemaker-template

S3_BUCKET_LF = 'ibdata-prod-ew1-s3-customer'
S3_BUCKET_LF_PREFIX = 'customer/load_factor_to_s3_nps_model'

S3_PATH_READ_NPS = 'customer/nps_surveys/export_historic'
S3_PATH_READ_LF = "customer/load_factor_to_s3_nps_model"

insert_date_ci='2024-06-11'
today_date_str='2024-06-11'

## Read data

In [5]:
# READ NPS DATA SOURCE
# Read df_nps_surveys
s3_resource = boto3.resource("s3")

# READ TODAY DATA (HISTORIC NPS)
today_nps_surveys_prefix = f'{S3_PATH_READ_NPS}/insert_date_ci={today_date_str}/'
s3_keys = [item.key for item in s3_resource.Bucket(S3_BUCKET_NPS).objects.filter(Prefix=today_nps_surveys_prefix)]
preprocess_paths = [f"s3://{S3_BUCKET_NPS}/{key}" for key in s3_keys]

df_nps_historic = pd.DataFrame()
for file in preprocess_paths:
    df = pd.read_csv(file)
    df_nps_historic = pd.concat([df_nps_historic, df], axis=0)
df_nps_historic = df_nps_historic.reset_index(drop=True)

In [6]:
# READ LF DATA SOURCE
# lf_dir = 's3://ibdata-prod-ew1-s3-customer/customer/load_factor_to_s3_nps_model/'    
load_factor_prefix = f's3://{S3_BUCKET_LF}/{S3_PATH_READ_LF}/'

# Assume rol for prod
sts_client = boto3.client('sts')
assumed_role = sts_client.assume_role(
    RoleArn="arn:aws:iam::320714865578:role/ibdata-prod-role-assume-customer-services-from-ibdata-aip-prod",
    RoleSessionName="test"
)
credentials = assumed_role['Credentials']
fs = s3fs.S3FileSystem(key=credentials['AccessKeyId'], secret=credentials['SecretAccessKey'], token=credentials['SessionToken'])

# Listall the files
load_factor_list = fs.ls(load_factor_prefix)
    
print("userlog: Read historic load_factor data path %s.", load_factor_prefix)
dataframes = []
for file_path in load_factor_list:
    try:
        file_info = fs.info(file_path)
        if file_info['Size'] == 0:
            print(f"Skipping empty file: {file_path}")
            continue

        with fs.open(f's3://{file_path}') as f:
            df = pd.read_csv(f)
            dataframes.append(df)
    except pd.errors.EmptyDataError:
        print(f"Caught EmptyDataError for file: {file_path}, skipping...")
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")

if dataframes:
    df_lf_historic = pd.concat(dataframes, ignore_index=True)
else:
    df_lf_historic = pd.DataFrame()

userlog: Read historic load_factor data path %s. s3://ibdata-prod-ew1-s3-customer/customer/load_factor_to_s3_nps_model/


## A little preprocess

In [7]:
condition_1 = (df_nps_historic['operating_airline_code'].isin(['IB', 'YW']))
condition_2 = ((df_nps_historic['invitegroup_ib'] != 3) | (df_nps_historic['invitegroup_ib'].isnull()))
condition_3 = (df_nps_historic['invitegroup'] == 2)

df_nps_historic = df_nps_historic.loc[condition_1 & (condition_2 & condition_3)]

df_lf_historic = df_lf_historic.loc[(df_lf_historic['operating_carrier'].isin(['IB', 'YW']))]

In [8]:
datetime_features = ['date_flight_local', 'scheduled_departure_time_local', 'scheduled_arrival_time_local', 'real_departure_time_local',
                     'real_arrival_time_local', 'started']
columns_to_cross_kpis=['cabin_in_surveyed_flight','haul']
columns_ext = ['tier_level', 'language_code', 'seat_no', 'volume_of_bags', 'number_of_child_in_the_booking', 'number_of_infant_in_the_booking',
              'number_of_people_in_the_booking', 'country_code', 'customer_journey_origin', 'customer_journey_destination', 'number_of_flights_in_journey',
              'order_of_flight_in_journey', 'marketing_airline_code', 'overall_haul', 'weight_category', 'ff_number', 'ticket_num', 'operating_airline_code',
               'nps_category', 'nps_100', 'group_age_survey', 'gender'] # invite_group

#'bkg_100_booking', 
touchpoints = ['bkg_200_journey_preparation', 'pfl_100_checkin', 'pfl_200_security', 'pfl_300_lounge',
               'pfl_500_boarding', 'ifl_300_cabin', 'ifl_200_flight_crew_annoucements', 'ifl_600_wifi', 'ifl_500_ife',
               'ifl_400_food_drink', 'ifl_100_cabin_crew', 'arr_100_arrivals', 'con_100_connections', 'pun_100_punctuality',
               'loy_200_loyalty_programme', 'inm_400_issues_response', 'img_310_ease_contact_phone']

# ,'img_320_ease_contact_ibplus_mail'
survey_fields = ['cla_600_wifi_t_f', 'tvl_journey_reason']

df_nps_historic['date_flight_local'] = pd.to_datetime(df_nps_historic['date_flight_local'])
df_lf_historic['flight_date_local'] = pd.to_datetime(df_lf_historic['flight_date_local'])

In [9]:
df_nps_historic = df_nps_historic[df_nps_historic['date_flight_local'].dt.year >= 2019]
df_nps_historic = df_nps_historic[~df_nps_historic['date_flight_local'].dt.year.isin([2020, 2021])]

df_lf_historic = df_lf_historic[~df_lf_historic['flight_date_local'].dt.year.isin([2020, 2021])]

In [10]:
delay_features = ['real_departure_time_local', 'scheduled_departure_time_local']
for feat in delay_features:
    df_nps_historic[feat] = pd.to_datetime(df_nps_historic[feat], format="%Y-%m-%d %H:%M:%S", errors = 'coerce')
            
df_nps_historic['delay_departure'] = (df_nps_historic['real_departure_time_local'] - df_nps_historic['scheduled_departure_time_local']).dt.total_seconds()/60

In [11]:
df_nps_historic['haul'] = df_nps_historic['haul'].replace('MH', 'SH')
#df_nps_historic['cabin_in_surveyed_flight'] = df_nps_historic['cabin_in_surveyed_flight'].replace('Premium Economy', 'Economy')# Load Factor
df_lf_historic['load_factor_business'] = df_lf_historic['pax_business'] / df_lf_historic['capacity_business']
df_lf_historic['load_factor_premium_ec'] = df_lf_historic['pax_premium_ec'] / df_lf_historic['capacity_premium_ec']
df_lf_historic['load_factor_economy'] = df_lf_historic['pax_economy'] / df_lf_historic['capacity_economy']

In [12]:
 # OTP
df_nps_historic['otp15_takeoff'] = (df_nps_historic['delay_departure'] > 15).astype(int)
df_nps_historic['otp30_takeoff'] = (df_nps_historic['delay_departure'] > 30).astype(int)
df_nps_historic['otp60_takeoff'] = (df_nps_historic['delay_departure'] > 60).astype(int)

# Promoter and Detractor columns
df_nps_historic["promoter_binary"] = df_nps_historic["nps_category"].apply(lambda x: 1 if x == "Promoter" else 0)
df_nps_historic["detractor_binary"] = df_nps_historic["nps_category"].apply(lambda x: 1 if x == "Detractor" else 0)

In [13]:
# Load Factor
df_lf_historic['load_factor_business'] = df_lf_historic['pax_business'] / df_lf_historic['capacity_business']
df_lf_historic['load_factor_premium_ec'] = df_lf_historic['pax_premium_ec'] / df_lf_historic['capacity_premium_ec']
df_lf_historic['load_factor_economy'] = df_lf_historic['pax_economy'] / df_lf_historic['capacity_economy']

In [14]:
cabin_to_load_factor_column = {
    'Economy': 'load_factor_economy',
    'Business': 'load_factor_business',
    'Premium Economy': 'load_factor_premium_ec'
}

# HISTORIC
df_lf_historic.columns = ['date_flight_local' if x=='flight_date_local' else 
                                'operating_airline_code' if x=='operating_carrier' else
                                'surveyed_flight_number' if x=='op_flight_num' else
                                x for x in df_lf_historic.columns]

df_historic = pd.merge(df_nps_historic, df_lf_historic, 
                    how='left', 
                    on=['date_flight_local', 'operating_airline_code', 'surveyed_flight_number', 'haul'])

df_historic['load_factor'] = df_historic.apply(lambda row: row[cabin_to_load_factor_column[row['cabin_in_surveyed_flight']]], axis=1)

In [15]:
df_historic['cabin_in_surveyed_flight']

0         Economy
1         Economy
2         Economy
3         Economy
4         Economy
           ...   
610240    Economy
610241    Economy
610242    Economy
610243    Economy
610244    Economy
Name: cabin_in_surveyed_flight, Length: 610245, dtype: object

In [16]:
df_nps_historic['delay_departure']

1          15.000
2          32.000
8           5.000
10          0.000
15         -1.000
            ...  
1074557    34.000
1074558    -3.000
1074559     6.000
1074560   -10.000
1074561     8.000
Name: delay_departure, Length: 605217, dtype: float64

In [17]:
df_historic['cabin_in_surveyed_flight'].unique()

array(['Economy', 'Business', 'Premium Economy'], dtype=object)

In [18]:
df_nps_historic['real_departure_time_local']

1         2023-01-13 07:45:00
2         2022-04-09 12:46:00
8         2022-08-28 20:29:00
10        2022-07-01 18:35:00
15        2022-04-23 12:13:00
                  ...        
1074557   2024-04-30 22:14:00
1074558   2024-04-29 06:27:00
1074559   2024-04-30 11:31:00
1074560   2024-04-30 11:30:00
1074561   2024-04-30 15:53:00
Name: real_departure_time_local, Length: 605217, dtype: datetime64[ns]

In [19]:
df_historic[df_historic['delay_departure']<0]['delay_departure'].min()

-1439.0

In [20]:
# Condition for dropping rows
condition = (df_historic['cabin_in_surveyed_flight'] == 'Premium Economy') & (df_historic['haul'] == 'SH')

# Keeping rows that do not meet the condition
df_historic = df_historic[~condition]

In [21]:
df_historic['respondent_id'].nunique()

605214

In [22]:
df_historic = df_historic.drop_duplicates(subset='respondent_id', keep='first')

In [36]:
filtered_df = df_historic[df_historic['delay'] != df_historic['delay_departure']]

In [38]:
filtered_df['date_flight_local']=pd.to_datetime(filtered_df['date_flight_local'])

In [39]:
df=filtered_df[filtered_df['date_flight_local'].dt.year>=2023]

In [40]:
df

,respondent_id,sample_id,surveyed_flight_number,date_flight_local,scheduled_departure_time_local,scheduled_arrival_time_local,tier_level,language_code,aircraft_registration_number,seat_no,volume_of_bags,number_of_child_in_the_booking,number_of_infant_in_the_booking,number_of_people_in_the_booking,infinita_customers_identifer,flag_of_ib_singular_customers,country_code,list_of_options_for_booking_channel,list_of_options_for_checkin_channel,lounge_used_at_origin_airport,customer_journey_origin,customer_journey_destination,number_of_flights_in_journey,order_of_flight_in_journey,fleet_in_surveyed_flight,marketing_airline_code,date_of_flight_gmt,scheduled_departure_time_gmt,real_departure_time_local,real_departure_time_gmt,scheduled_arrival_time_gmt,real_arrival_time_local,real_arrival_time_gmt,segment,route,overall_haul,purser,invitegroup_ib,weight_category,weekly_weight,monthly_weight,pnr_show,ff_number,id_golden_record,ticket_num,started,time_spent_hrminsec,customer_email_show,origin_of_surveyed_flight,destination_of_surveyed_flight,operating_airline_code,cabin_in_surveyed_flight,haul,first_name_show,last_name_show,second_last_name_show,nps_category,nps_100,survey_type,invitegroup,group_age_survey,pun_100_punctuality,inm_200_issues_prior_checkin,inm_200_issues_prior_ticket_change,inm_200_issues_prior_schedule_change,inm_200_issues_prior_contact_center,inm_200_issues_prior_special_serv,inm_200_issues_prior_special_req,inm_200_issues_prior_avios,inm_200_issues_prior_voucher,inm_206_issues_checkin_long_queues,inm_206_issues_checkin_wrong_info,inm_206_issues_checkin_additional_fees,inm_206_issues_checkin_overbooking,inm_206_issues_checkin_downgrade,inm_206_issues_checkin_staff,inm_206_issues_checkin_social_distance,inm_206_issues_checkin_face_masks,inm_206_issues_checkin_documentation,inm_206_issues_checkin_other,inm_207_issues_lounge_denied,inm_207_issues_lounge_overcrowded,inm_207_issues_lounge_cleanliness,inm_207_issues_lounge_wifi,inm_207_issues_lounge_staff,inm_207_issues_lounge_food_drink,inm_207_issues_lounge_face_masks,inm_207_issues_lounge_other,inm_208_issues_security_leave_sth,inm_208_issues_security_long_queues,inm_208_issues_security_staff,inm_208_issues_security_social_distance,inm_208_issues_security_face_masks,inm_208_issues_security_other,inm_209_issues_boarding_unclear,inm_209_issues_boarding_gate_changed,inm_209_issues_boarding_lack_space,inm_209_issues_boarding_disorganised,inm_209_issues_boarding_staff,inm_209_issues_boarding_social_distance,inm_209_issues_boarding_face_masks,inm_209_issues_boarding_documentation,inm_209_issues_boarding_other,inm_220_issues_timing_cancelled,inm_220_issues_timing_dep_delay,inm_220_issues_timing_arr_delay,inm_220_issues_timing_missed,inm_230_issues_onboard_staff,inm_230_issues_onboard_ife,inm_230_issues_onboard_overcrowding,inm_230_issues_onboard_face_masks,inm_235_issues_onboard_comfort_damaged,inm_235_issues_onboard_comfort_space,inm_235_issues_onboard_comfort_temperature,inm_235_issues_onboard_comfort_cleanliness,inm_235_issues_onboard_comfort_washrooms,inm_235_issues_onboard_comfort_other,inm_236_issues_meal_availability,inm_236_issues_meal_portions,inm_236_issues_meal_quality,inm_236_issues_meal_special,inm_236_issues_meal_other,inm_240_issues_baggage_lost,inm_240_issues_baggage_delayed,inm_240_issues_baggage_demaged,inm_240_issues_baggage_staff,inm_240_issues_baggage_hand,inm_240_issues_baggage_other,inm_250_issues_arrival_slow,inm_250_issues_arrival_unclear,inm_250_issues_arrival_aditional_request,inm_250_issues_arrival_staff,inm_250_issues_arrival_immigration_queues,inm_250_issues_arrival_immigration_passport,inm_250_issues_arrival_immigration_other,inm_255_issues_connecting_missed,inm_255_issues_connecting_staff,inm_255_issues_connecting_baggage,inm_255_issues_connecting_other,bkg_100_booking,bkg_200_journey_preparation,inm_400_issues_response,pfl_100_checkin,pfl_200_security,pfl_300_lounge,pfl_500_boarding,ifl_100_cabin_crew,ifl_200_flight_crew_annoucements,ifl_300_cabin,

In [23]:
import pandas as pd

# Assuming df_historic is your DataFrame and it has been properly imported
respondent_ids = [64986539, 64987164, 65097632, 64890118, 64642526]
filtered_df = df_historic[df_historic['respondent_id'].isin(respondent_ids)]
filtered_df[['respondent_id', 'otp15_takeoff', 'delay_departure']]

In [32]:
check = pd.read_csv('predictions (9).csv')
filtered_df = check[check['respondent_id'].isin(respondent_ids)]
filtered_df[['respondent_id', 'otp15_takeoff']]

,respondent_id,otp15_takeoff


In [30]:
filtered_df[['respondent_id', 'otp15_takeoff', 'delay_departure', 'date_flight_local']]

,respondent_id,otp15_takeoff,delay_departure,date_flight_local
198724,64642526,1,16.000,2024-02-09
276898,64987164,1,47.000,2024-02-25
381605,65097632,1,18.000,2024-02-29
394195,64986539,1,36.000,2024-02-25
592081,64890118,1,194.000,2024-02-20


In [26]:
[col for col in df_historic.columns if 'issues' in col]

['inm_200_issues_prior_checkin',
 'inm_200_issues_prior_ticket_change',
 'inm_200_issues_prior_schedule_change',
 'inm_200_issues_prior_contact_center',
 'inm_200_issues_prior_special_serv',
 'inm_200_issues_prior_special_req',
 'inm_200_issues_prior_avios',
 'inm_200_issues_prior_voucher',
 'inm_206_issues_checkin_long_queues',
 'inm_206_issues_checkin_wrong_info',
 'inm_206_issues_checkin_additional_fees',
 'inm_206_issues_checkin_overbooking',
 'inm_206_issues_checkin_downgrade',
 'inm_206_issues_checkin_staff',
 'inm_206_issues_checkin_social_distance',
 'inm_206_issues_checkin_face_masks',
 'inm_206_issues_checkin_documentation',
 'inm_206_issues_checkin_other',
 'inm_207_issues_lounge_denied',
 'inm_207_issues_lounge_overcrowded',
 'inm_207_issues_lounge_cleanliness',
 'inm_207_issues_lounge_wifi',
 'inm_207_issues_lounge_staff',
 'inm_207_issues_lounge_food_drink',
 'inm_207_issues_lounge_face_masks',
 'inm_207_issues_lounge_other',
 'inm_208_issues_security_leave_sth',
 'inm_20

In [ ]:
def plot_variable_correlations(corr_data, target_var):
    # Create correlation-specific graphs with otp15_takeoff
    # methods = ['pearson', 'spearman', 'kendall']
    methods = ['pearson']
    for method in methods:
        corr = corr_data.corr(method=method)
        target_corr = corr[[target_var]].sort_values(by=target_var, ascending=False)

        plt.figure(figsize=(12, 8))
        sns.barplot(y=target_corr.index, x=target_corr[target_var], palette='coolwarm')
        plt.title(f'{method.capitalize()} Correlation with {target_var}')
        plt.xlabel(f'{method.capitalize()} Correlation Coefficient')
        plt.ylabel('Variables')
        plt.tight_layout()
        plt.show()
def scatter_plot(df, variable, target):# Create the scatter plot
    plt.figure(figsize=(10, 6))
    plt.scatter(df[variable], df[target], alpha=0.6, edgecolors='w', linewidths=0.5)
    plt.title(f'{variable} vs {target}')
    plt.xlabel(f'{variable}')
    plt.ylabel(f'{target}')
    plt.grid(True)
    plt.show()

In [ ]:
corr_data = df_historic[['pun_100_punctuality','delay_departure', 'nps_100', 'date_flight_local']]

In [ ]:
cabin= "Economy"
haul= "SH"
corr_data = df_historic[(df_historic['cabin_in_surveyed_flight']==cabin) & (df_historic['haul']==haul)][['pun_100_punctuality','delay_departure', 'date_flight_local',  'nps_100']]

In [ ]:
target= 'delay_departure'
variable = 'date_flight_local'

corr_data['date_flight_local']=pd.to_datetime(corr_data['date_flight_local'])
# plot_variable_correlations(corr_data, 'nps_100')
scatter_plot(corr_data, variable, target)

In [ ]:
target= 'pun_100_punctuality'
variable = 'delay_departure'
corr_data['delay_departure'] = corr_data['delay_departure'].clip(lower=0)
plot_variable_correlations(corr_data, 'nps_100')
scatter_plot(corr_data, variable, target)

## Correlation analysis

In [ ]:
df_issues = df_historic[df_historic['inm_050_issues_t_f']=='Yes'].copy()

In [ ]:
df_no_issues = df_historic[df_historic['inm_050_issues_t_f']=='No'].copy()

In [ ]:
df_issues['respondent_id'].nunique()

In [ ]:
df_no_issues['respondent_id'].nunique()

In [ ]:
def calculate_nps(promoters, detractors, total_responses):
    """Calcula el Net Promoter Score (NPS)."""
    if total_responses == 0:
        return np.nan
    return ((promoters - detractors) / total_responses) * 100

def calculate_weighted_nps(group_df):
    """Calcula el NPS ponderado para un grupo de datos."""
    promoters_weight = group_df.loc[group_df['nps_100'] > 8, 'monthly_weight'].sum()
    detractors_weight = group_df.loc[group_df['nps_100'] <= 6, 'monthly_weight'].sum()
    total_weight = group_df['monthly_weight'].sum()
    
    if total_weight == 0:
        return np.nan
    return (promoters_weight - detractors_weight) / total_weight * 100

def calculate_satisfaction(df, variable):
    """Calcula la tasa de satisfacción para una variable dada, utilizando pesos mensuales si están disponibles."""
    # Comprobar si la columna 'monthly_weight' existe y no está completamente vacía para los datos relevantes
    if 'monthly_weight' in df.columns and not df[df[variable].notnull()]['monthly_weight'].isnull().all():
        # Suma de los pesos donde la variable es >= 8 y satisface la condición de estar satisfecho
        satisfied_weight = df[df[variable] >= 8]['monthly_weight'].sum()
        # Suma de todos los pesos donde la variable no es NaN
        total_weight = df[df[variable].notnull()]['monthly_weight'].sum()
        # Calcula el porcentaje de satisfacción usando los pesos
        if total_weight == 0:
            return np.nan
        return (satisfied_weight / total_weight) * 100
    else:
        # Contar respuestas satisfechas
        satisfied_count = df[df[variable] >= 8].shape[0]
        # Contar total de respuestas válidas
        total_count = df[variable].notnull().sum()
        # Calcula el porcentaje de satisfacción usando conteo
        if total_count == 0:
            return np.nan
        return (satisfied_count / total_count) * 100




def calculate_otp(df, n):
    """Calcula el On-Time Performance (OTP) como el porcentaje de valores igual a 1."""
    on_time_count = (df[f'otp{n}_takeoff'] == 0).sum()
    total_count = df[f'otp{n}_takeoff'].notnull().sum()
    return (on_time_count / total_count) * 100 if total_count > 0 else 0


def calculate_load_factor(df, pax_column, capacity_column):
    """Calcula el factor de carga para una cabina específica."""
    total_pax = df[pax_column].sum()
    total_capacity = df[capacity_column].sum()
    # Evitar la división por cero
    if total_capacity > 0:
        return (total_pax / total_capacity) * 100
    else:
        return 0

    
def calculate_metrics_summary(df, start_date, end_date, touchpoints):
    # Filtrar por rango de fechas
    df_filtered = df[(df['date_flight_local'] >= pd.to_datetime(start_date)) & (df['date_flight_local'] <= pd.to_datetime(end_date))]
    
    # Mapeo de cabinas a columnas de pax y capacidad
    cabin_mapping = {
        'Economy': ('pax_economy', 'capacity_economy'),
        'Business': ('pax_business', 'capacity_business'),
        'Premium Economy': ('pax_premium_ec', 'capacity_premium_ec')
    }
    
    results_list = []
    
    for (cabin, haul), group_df in df_filtered.groupby(['cabin_in_surveyed_flight', 'haul']):
        
        print(f'CABIN/HAUL: {cabin}/{haul}')
        result = {
            'start_date': start_date,
            'end_date': end_date,
            'cabin_in_surveyed_flight': cabin,
            'haul': haul,
            'otp15_takeoff': calculate_otp(group_df, 15),
            'otp30_takeoff': calculate_otp(group_df, 30),
            'otp60_takeoff': calculate_otp(group_df, 60),
            'mean_delay': group_df[group_df['delay_departure']>0]['delay_departure'].mean()
        }
        
        # Calcula el NPS para el grupo
        promoters = (group_df['nps_100'] >= 9).sum()
        detractors = (group_df['nps_100'] <= 6).sum()
        total_responses = group_df['nps_100'].notnull().sum()
        result['NPS'] = calculate_nps(promoters, detractors, total_responses) if total_responses else None
        
        # Calcula el NPS ponderado para el grupo
        result['NPS_weighted'] = calculate_weighted_nps(group_df)
        
        # Satisfacción para cada touchpoint
        for tp in touchpoints:
            result[f'{tp}_satisfaction'] = calculate_satisfaction(group_df, tp)
            
        
        # Calcula el factor de carga para la cabina
        pax_column, capacity_column = cabin_mapping.get(cabin, (None, None))
        if pax_column and capacity_column:
            result['load_factor'] = calculate_load_factor(group_df, pax_column, capacity_column)
        
        results_list.append(result)
    
    return pd.DataFrame(results_list)

def generate_date_intervals(start_date, end_date, freq=1):
    """Genera una lista de tuplas con intervalos de fechas desde start_date hasta end_date."""
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    intervals = []
    while start_date < end_date:
        intervals.append((start_date, start_date))
        start_date = start_date + pd.Timedelta(days=freq)
    return intervals

def calculate_metrics_for_intervals(df, touchpoints, start_date, end_date, freq):
    """Calcula las métricas para todos los intervalos posibles hasta end_date."""
    intervals = generate_date_intervals(start_date, end_date, freq)
    all_metrics = []

    for interval_start, interval_end in intervals:
        interval_metrics = calculate_metrics_summary(df, interval_start, interval_end, touchpoints)
        print(f"Interval: {interval_start} to {interval_end}, Data points: {len(interval_metrics)}")
        all_metrics.append(interval_metrics)

    
    # Concatenar todos los DataFrames de resultados en uno solo
    results_df = pd.concat(all_metrics, ignore_index=True)
    return results_df


In [ ]:
# all_weekly = calculate_metrics_for_intervals(df_historic, touchpoints, '01-01-2022', '04-06-2024', 7)
# issues_weekly = calculate_metrics_for_intervals(df_issues, touchpoints, '01-01-2022', '04-06-2024', 7)
# no_issues_weekly = calculate_metrics_for_intervals(df_no_issues, touchpoints, '01-01-2022', '04-06-2024', 7)

In [ ]:
df_historic['delay_departure'] = df_historic['delay_departure'].clip(lower=0)
all_daily = calculate_metrics_for_intervals(df_historic, touchpoints, '01-01-2022', '04-06-2024', 1)
# issues_daily = calculate_metrics_for_intervals(df_issues, touchpoints, '01-01-2022', '04-06-2024', 1)
# no_issues_daily = calculate_metrics_for_intervals(df_no_issues, touchpoints, '01-01-2022', '04-06-2024', 1)

In [ ]:
import pandas as pd

# Assuming all_weekly, issues_weekly, no_issues_weekly are already defined
dataframes_daily = {
    'all_daily': all_daily,
    # 'issues_daily': issues_daily,
    # 'no_issues_daily': no_issues_daily
}

# dataframes_weekly = {
#     'all_weekly': all_weekly,
#     'issues_weekly': issues_weekly,
#     'no_issues_weekly': no_issues_weekly
# }

# Define the function to filter and select columns
def filter_and_select(df, cabin, haul):
    filtered_df = df[(df['cabin_in_surveyed_flight'] == cabin) & (df['haul'] == haul)]
    cols = [col for col in filtered_df.columns if '_satisfaction' in col] + ['otp15_takeoff', 'otp30_takeoff', 'otp60_takeoff', 'mean_delay', 'load_factor', 'NPS_weighted']
    return filtered_df[cols]

# Dictionary to hold the results
daily_result_dict = {key: {} for key in dataframes_daily.keys()}

# Loop through each DataFrame and unique combinations
for name, df in dataframes_daily.items():
    unique_combinations = df[['cabin_in_surveyed_flight', 'haul']].drop_duplicates()
    for _, row in unique_combinations.iterrows():
        result_df = filter_and_select(df, row['cabin_in_surveyed_flight'], row['haul'])
        cabin_haul_key = f"{row['cabin_in_surveyed_flight']}_{row['haul']}"
        daily_result_dict[name][cabin_haul_key] = result_df
        
# Dictionary to hold the results
# weekly_result_dict = {key: {} for key in dataframes_weekly.keys()}

# # Loop through each DataFrame and unique combinations
# for name, df in dataframes_weekly.items():
#     unique_combinations = df[['cabin_in_surveyed_flight', 'haul']].drop_duplicates()
#     for _, row in unique_combinations.iterrows():
#         result_df = filter_and_select(df, row['cabin_in_surveyed_flight'], row['haul'])
#         cabin_haul_key = f"{row['cabin_in_surveyed_flight']}_{row['haul']}"
#         weekly_result_dict[name][cabin_haul_key] = result_df

In [ ]:
daily_result_dict

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import mutual_info_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
import shap

def plot_variable_correlations(corr_data, target_var):
    # Create correlation-specific graphs with otp15_takeoff
    # methods = ['pearson', 'spearman', 'kendall']
    methods = ['pearson']
    for method in methods:
        corr = corr_data.corr(method=method)
        target_corr = corr[[target_var]].sort_values(by=target_var, ascending=False)

        plt.figure(figsize=(12, 8))
        sns.barplot(y=target_corr.index, x=target_corr[target_var], palette='coolwarm')
        plt.title(f'{method.capitalize()} Correlation with {target_var}')
        plt.xlabel(f'{method.capitalize()} Correlation Coefficient')
        plt.ylabel('Variables')
        plt.tight_layout()
        plt.show()

def analyze_correlations(corr_data, name):
    # Calculate correlations
    pearson_corr = corr_data.corr(method='pearson')
    spearman_corr = corr_data.corr(method='spearman')
    kendall_corr = corr_data.corr(method='kendall')

    # Pearson Correlation Matrix visualization
    # plt.figure(figsize=(20, 16))
    # sns.heatmap(pearson_corr, annot=True, fmt=".2f", cmap='coolwarm', cbar=True, linewidths=0.5, linecolor='gray')
    # plt.title('Pearson Correlation Matrix for ' + name)
    # plt.xticks(rotation=45, ha='right')
    # plt.yticks(rotation=0)
    # plt.tight_layout()
    # plt.show()

    # Spearman Correlation Matrix visualization
#     plt.figure(figsize=(20, 16))
#     sns.heatmap(spearman_corr, annot=True, fmt=".2f", cmap='coolwarm', cbar=True, linewidths=0.5, linecolor='gray')
#     plt.title('Spearman Correlation Matrix for ' + name)
#     plt.xticks(rotation=45, ha='right')
#     plt.yticks(rotation=0)
#     plt.tight_layout()
#     plt.show()

#     # Kendall Correlation Matrix visualization
#     plt.figure(figsize=(20, 16))
#     sns.heatmap(kendall_corr, annot=True, fmt=".2f", cmap='coolwarm', cbar=True, linewidths=0.5, linecolor='gray')
#     plt.title('Kendall Correlation Matrix for ' + name)
#     plt.xticks(rotation=45, ha='right')
#     plt.yticks(rotation=0)
#     plt.tight_layout()
#     plt.show()
    
    # Call the function to plot correlations with otp15_takeoff
    plot_variable_correlations(corr_data, 'NPS_weighted')

    
    # Check if any column is entirely NaN and fill or drop accordingly
    if corr_data.isnull().all().any():
        # Option 1: Drop columns that are completely NaN
        corr_data = corr_data.dropna(axis=1, how='all')
        # Option 2: Fill completely NaN columns with a placeholder if dropping is not desired
        # corr_data = corr_data.fillna(value={col: 0 for col in corr_data.columns if corr_data[col].isnull().all()})
        
    # Impute NaNs
    imputer = SimpleImputer(strategy='median')
    corr_data_imputed = pd.DataFrame(imputer.fit_transform(corr_data), columns=corr_data.columns)

    # Feature and target separation
    X = corr_data_imputed.drop(columns=['NPS_weighted'])
    y = corr_data_imputed['NPS_weighted']

    # Calculate Mutual Information
    mi = mutual_info_regression(X, y, random_state=42)
    mi_df = pd.DataFrame(mi, index=X.columns, columns=['Mutual Information']).sort_values(by='Mutual Information', ascending=False)

    # Mutual Information visualization
    plt.figure(figsize=(12, 8))
    sns.barplot(x=mi_df['Mutual Information'], y=mi_df.index, palette='viridis')
    plt.title('Mutual Information between NPS_weighted and Other Variables for ' + name)
    plt.xlabel('Mutual Information')
    plt.ylabel('Features')
    plt.tight_layout()
    plt.show()

    # Save results to an Excel file
    with pd.ExcelWriter('correlation_analysis_results_' + name + '.xlsx') as writer:
        pearson_corr.to_excel(writer, sheet_name='Pearson Correlation')
        spearman_corr.to_excel(writer, sheet_name='Spearman Correlation')
        kendall_corr.to_excel(writer, sheet_name='Kendall Correlation')
        mi_df.to_excel(writer, sheet_name='Mutual Information')


In [ ]:
def scatter_plot(df, variable, target):# Create the scatter plot
    plt.figure(figsize=(10, 6))
    plt.scatter(df[variable], df[target], alpha=0.6, edgecolors='w', linewidths=0.5)
    plt.title(f'{variable} vs {target}')
    plt.xlabel(f'{variable}')
    plt.ylabel(f'{target}')
    plt.grid(True)
    plt.show()

In [ ]:
# Apply the analysis function to each DataFrame in the result_dict
for name, dfs in daily_result_dict.items():
    print(name)
    for cabin_haul_key, df in dfs.items():
        print(cabin_haul_key)
        if not df.empty:
            analyze_correlations(df, name + '_' + cabin_haul_key)
            scatter_plot(df, 'ifl_100_cabin_crew_satisfaction', 'otp15_takeoff')
            scatter_plot(df, 'con_100_connections_satisfaction', 'otp15_takeoff')
            

In [ ]:
# Apply the analysis function to each DataFrame in the result_dict
for name, dfs in weekly_result_dict.items():
    print(name)
    for cabin_haul_key, df in dfs.items():
        print(cabin_haul_key)
        if not df.empty:
            analyze_correlations(df, name + '_' + cabin_haul_key)

## Agregation logic

Given a date it takes it as an "end_date" and computes every interval with previous dates. Then it perfomr the satisfaction, NPS, load factor and otp aggregations for that particular interval.

In [ ]:
# Funciones auxiliares
import numpy as np

def calculate_nps(promoters, detractors, total_responses):
    """Calcula el Net Promoter Score (NPS)."""
    if total_responses == 0:
        return np.nan
    return ((promoters - detractors) / total_responses) * 100

def calculate_weighted_nps(group_df):
    """Calcula el NPS ponderado para un grupo de datos."""
    promoters_weight = group_df.loc[group_df['nps_100'] > 8, 'monthly_weight'].sum()
    detractors_weight = group_df.loc[group_df['nps_100'] <= 6, 'monthly_weight'].sum()
    total_weight = group_df['monthly_weight'].sum()
    
    if total_weight == 0:
        return np.nan
    return (promoters_weight - detractors_weight) / total_weight * 100

def calculate_satisfaction(df, variable):
    """Calcula la tasa de satisfacción para una variable dada, utilizando pesos mensuales si están disponibles."""
    # Comprobar si la columna 'monthly_weight' existe y no está completamente vacía para los datos relevantes
    if 'monthly_weight' in df.columns and not df[df[variable].notnull()]['monthly_weight'].isnull().all():
        # Suma de los pesos donde la variable es >= 8 y satisface la condición de estar satisfecho
        satisfied_weight = df[df[variable] >= 8]['monthly_weight'].sum()
        # Suma de todos los pesos donde la variable no es NaN
        total_weight = df[df[variable].notnull()]['monthly_weight'].sum()
        # Calcula el porcentaje de satisfacción usando los pesos
        if total_weight == 0:
            return np.nan
        return (satisfied_weight / total_weight) * 100
    else:
        # Contar respuestas satisfechas
        satisfied_count = df[df[variable] >= 8].shape[0]
        # Contar total de respuestas válidas
        total_count = df[variable].notnull().sum()
        # Calcula el porcentaje de satisfacción usando conteo
        if total_count == 0:
            return np.nan
        return (satisfied_count / total_count) * 100




def calculate_otp(df, variable='otp15_takeoff'):
    """Calcula el On-Time Performance (OTP) como el porcentaje de valores igual a 1."""
    on_time_count = (df[variable] == 0).sum()
    total_count = df[variable].notnull().sum()
    return (on_time_count / total_count) * 100 if total_count > 0 else 0


def calculate_load_factor(df, pax_column, capacity_column):
    """Calcula el factor de carga para una cabina específica."""
    total_pax = df[pax_column].sum()
    total_capacity = df[capacity_column].sum()
    # Evitar la división por cero
    if total_capacity > 0:
        return (total_pax / total_capacity) * 100
    else:
        return 0

    
def calculate_metrics_summary(df, start_date, end_date, touchpoints):
    # Filtrar por rango de fechas
    df_filtered = df[(df['date_flight_local'] >= pd.to_datetime(start_date)) & (df['date_flight_local'] <= pd.to_datetime(end_date))]
    
    # Mapeo de cabinas a columnas de pax y capacidad
    cabin_mapping = {
        'Economy': ('pax_economy', 'capacity_economy'),
        'Business': ('pax_business', 'capacity_business'),
        'Premium Economy': ('pax_premium_ec', 'capacity_premium_ec')
    }
    
    results_list = []
    
    for (cabin, haul), group_df in df_filtered.groupby(['cabin_in_surveyed_flight', 'haul']):
        
        print(f'CABIN/HAUL: {cabin}/{haul}')
        result = {
            'start_date': start_date,
            'end_date': end_date,
            'cabin_in_surveyed_flight': cabin,
            'haul': haul,
            'otp15_takeoff': calculate_otp(group_df)
        }
        
        # Calcula el NPS para el grupo
        promoters = (group_df['nps_100'] >= 9).sum()
        detractors = (group_df['nps_100'] <= 6).sum()
        total_responses = group_df['nps_100'].notnull().sum()
        result['NPS'] = calculate_nps(promoters, detractors, total_responses) if total_responses else None
        
        # Calcula el NPS ponderado para el grupo
        result['NPS_weighted'] = calculate_weighted_nps(group_df)
        
        # Satisfacción para cada touchpoint
        for tp in touchpoints:
            result[f'{tp}_satisfaction'] = calculate_satisfaction(group_df, tp)
            
        
        # Calcula el factor de carga para la cabina
        pax_column, capacity_column = cabin_mapping.get(cabin, (None, None))
        if pax_column and capacity_column:
            result['load_factor'] = calculate_load_factor(group_df, pax_column, capacity_column)
        
        results_list.append(result)
    
    return pd.DataFrame(results_list)

def generate_date_intervals(start_date, end_date):
    """Genera una lista de tuplas con intervalos de fechas desde start_date hasta end_date."""
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    intervals = [(start_date + pd.Timedelta(days=d), end_date) for d in range((end_date - start_date).days + 1)]
    return intervals

def calculate_metrics_for_intervals(df, touchpoints, start_date, end_date):
    """Calcula las métricas para todos los intervalos posibles hasta end_date."""
    intervals = generate_date_intervals(start_date, end_date)
    all_metrics = []

    for interval_start, interval_end in intervals:
        interval_metrics = calculate_metrics_summary(df, interval_start, interval_end, touchpoints)
        print(f"Interval: {interval_start} to {interval_end}, Data points: {len(interval_metrics)}")
        all_metrics.append(interval_metrics)

    
    # Concatenar todos los DataFrames de resultados en uno solo
    results_df = pd.concat(all_metrics, ignore_index=True)
    return results_df

# # Ejemplo de uso:
# # touchpoints = ['tp1', 'tp2', 'tp3']  # Asegúrate de reemplazar estos con los nombres reales de tus touchpoints
# df_result = calculate_metrics_summary(df_historic, '2023-01-01', '2023-01-31', touchpoints)
# # print(df_result)

# # Definir la fecha de inicio del año y la fecha de fin específica
# start_date = '2023-03-01'
# end_date = '2023-05-01'



# results_intervals_df = calculate_metrics_for_intervals(df_historic, touchpoints, start_date, end_date)




In [ ]:
results_intervals_df

In [ ]:
all_intervals_results

In [ ]:
all_intervals_results.to_csv('intervals.csv')

# 2. Prediction with Darts model

In [ ]:
all_weekly

In [ ]:
df_all = df_historic[(df_historic['cabin_in_surveyed_flight']=='Economy') & (df_historic['haul']=='SH')]

In [ ]:
df_issues_BLH = df_issues[(df_issues['cabin_in_surveyed_flight']=='Economy') & (df_issues['haul']=='SH')]

In [ ]:
df_no_issues_BLH = df_no_issues[(df_no_issues['cabin_in_surveyed_flight']=='Economy') & (df_no_issues['haul']=='SH')]

In [ ]:
df_all_agg = calculate_metrics_summary(df_all, '2024-01-01', '2024-05-24', touchpoints)
issues_BLH = calculate_metrics_summary(df_issues_BLH, '2024-01-01', '2024-05-24', touchpoints)
no_issues_BLH = calculate_metrics_summary(df_no_issues_BLH, '2024-01-01', '2024-05-24', touchpoints)

In [ ]:
issues_BLH

In [ ]:

# Concatenate the DataFrames
concatenated_df = pd.concat([issues_BLH, no_issues_BLH], ignore_index=True)
concatenated_df['insert_date_ci']='2024-06-01'

# Display the concatenated DataFrame
print(concatenated_df)

In [ ]:
import pkg_resources

# List of libraries you want to check versions for
libraries = [
    "s3fs", "boto", "boto3", "botocore", "numpy", "scikit-image",
    "scikit-learn", "scipy", "PyYAML", "pandas", "darts",
    "optuna", "shap", "lightgbm"
]

# Check the installed version for each library and print it
for library in libraries:
    try:
        version = pkg_resources.get_distribution(library).version
        print(f"{library}: {version}")
    except pkg_resources.DistributionNotFound:
        print(f"{library} is not installed.")


In [ ]:
import darts
from darts import TimeSeries
from darts.utils.timeseries_generation import (
    gaussian_timeseries,
    linear_timeseries,
    sine_timeseries,
)

from darts.metrics import mape, smape, mae
from darts.dataprocessing.transformers import Scaler
from darts.utils.timeseries_generation import datetime_attribute_timeseries

from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import RandomForestRegressor

import lightgbm

from darts.models import LightGBMModel

from darts.models import LightGBMModel, RandomForest, LinearRegressionModel
from darts.utils.statistics import check_seasonality, plot_acf, plot_residuals_analysis

from darts.explainability.shap_explainer import ShapExplainer
import pickle
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from darts.models import LinearRegressionModel, LightGBMModel, RandomForest
from calendar import month_name as mn
import os

import shap
import pandas as pd

In [ ]:
year_targets_df = pd.read_csv('operative_performance_corrected_annual_targets_2023-11-21.csv')


In [ ]:
year_targets_df.rename(columns={
    'cabin' : 'cabin_in_surveyed_flight',
    'date_flight_local': 'start_date'  # Assuming you want to consider end_date as the equivalent of date_flight_local
}, inplace=True)

# Correct the conversion to datetime objects
year_targets_df['start_date'] = pd.to_datetime(year_targets_df['start_date'])

# Compute 'start_date' as the first day of the corresponding month
# Using dt.to_period('M').to_timestamp() to safely navigate datetime formats
year_targets_df['end_date'] = year_targets_df['start_date'] + pd.offsets.YearEnd(0)

In [ ]:
satisfaction_cols = [col for col in year_targets_df.columns if col.endswith('_satisfaction')]
otp_cols = ['otp15_takeoff']
features_cols = satisfaction_cols + ['load_factor'] + otp_cols

In [ ]:
year_targets_df['insert_date_ci']='2023-11-21'

In [ ]:
year_targets_df=year_targets_df[['start_date', 'end_date','cabin_in_surveyed_flight','haul','otp15_takeoff', 'NPS', 'NPS_weighted']+ satisfaction_cols + ['load_factor','insert_date_ci']]

In [ ]:
year_targets_df= year_targets_df[(year_targets_df['start_date']=='2023-12-31')]

In [ ]:
year_targets_df

In [ ]:
# Make a copy of the first two rows
copy_df = year_targets_df[(year_targets_df['haul']=='SH')].copy()

# Add 0.5 to the 'otp15_takeoff' column in the copied DataFrame
copy_df['otp15_takeoff'] += 0.5

copy_df['insert_date_ci'] = '2024-05-08'


In [ ]:
corr_data=pd.read_csv('data_for_historic_prediction (5).csv')

In [ ]:
corr_data['start_date'].min()

In [ ]:
satisfaction_cols = [col for col in corr_data.columns if col.endswith('_satisfaction')]
otp_cols = ['otp15_takeoff']
features_cols = satisfaction_cols + ['load_factor'] + otp_cols

In [ ]:
buss_corr = corr_data[(corr_data['cabin_in_surveyed_flight'] == 'Business') & (corr_data['haul'] == 'SH')][features_cols].corr()
eco_corr = corr_data[(corr_data['cabin_in_surveyed_flight'] == 'Economy') & (corr_data['haul'] == 'SH')][features_cols].corr()


# Splitting the DataFrame
df_business = year_targets_df[(year_targets_df['cabin_in_surveyed_flight'] == 'Business') & (year_targets_df['haul']=='SH')].copy()
df_economy = year_targets_df[(year_targets_df['cabin_in_surveyed_flight'] == 'Economy') & (year_targets_df['haul']=='SH')].copy()

# Function to adjust values based on the top 5 correlated features and correlation matrix
def adjust_top_5_values(df, correlation_matrix, delta_change):
    # Identify the top 5 features based on absolute correlation with 'otp15_takeoff'
    top_features = correlation_matrix['otp15_takeoff'].abs().nlargest(5).index

    # Apply adjustments only to the top 5 features
    for column in top_features:
        if column != 'otp15_takeoff':  # Ensure we're not adjusting 'otp15_takeoff' again
            adjustment_factor = correlation_matrix.at[column, 'otp15_takeoff'] * delta_change
            print(f"Adjustment_factor for column {column}: {adjustment_factor}")
            df[column] += adjustment_factor

    return df

# Sample usage with your existing DataFrame splits
delta_otp15_takeoff = 0.5
df_business['otp15_takeoff'] += delta_otp15_takeoff
df_economy['otp15_takeoff'] += delta_otp15_takeoff

# Assuming 'buss_corr' and 'eco_corr' are defined as your business and economy correlation matrices
print('Business')
df_business = adjust_top_5_values(df_business, buss_corr, delta_otp15_takeoff)
print('Economy')
df_economy = adjust_top_5_values(df_economy, eco_corr, delta_otp15_takeoff)

# Optionally recombine the DataFrames and set new insert dates
updated_df = pd.concat([df_business, df_economy]).sort_index()
updated_df['insert_date_ci'] = '2024-05-08'  # Update insert date for all




In [ ]:
updated_df

In [ ]:
concatenated_df=pd.concat([year_targets_df,updated_df], ignore_index=True)

In [ ]:
day_predict_df = pd.read_csv('intervals.csv')

In [ ]:
concatenated_df

In [ ]:
import pandas as pd

In [ ]:
targets_df = pd.read_csv('targets.csv')

targets_df=targets_df[targets_df['cabin']!='Global']

targets_df['insert_date_ci']='2023-11-21'

targets_df.rename(columns={
    'cabin' : 'cabin_in_surveyed_flight',
    'date_flight_local': 'end_date'  # Assuming you want to consider end_date as the equivalent of date_flight_local
}, inplace=True)
# day_predict_df.rename(columns={
#     'interval_end_date': 'insert_date_ci'  # Assuming you want to consider end_date as the equivalent of date_flight_local
# }, inplace=True)

# Correct the conversion to datetime objects
targets_df['end_date'] = pd.to_datetime(targets_df['end_date'])

targets_df = targets_df[(targets_df['end_date'].dt.year == 2024)]

# Compute 'start_date' as the first day of the corresponding month
# Using dt.to_period('M').to_timestamp() to safely navigate datetime formats
targets_df['start_date'] = targets_df['end_date'].dt.to_period('M').dt.to_timestamp()

targets_df.drop(columns=['Unnamed: 0'], inplace=True)




In [ ]:
# Monthly improvements
month_improvements = {
    1: 0.7, 2: 0.2, 3: 0.5, 4: 0.3, 5: 0.2, 6: 0.3,
    7: 0.6, 8: 0.6, 9: 0.8, 10: 0.7, 11: 0.5, 12: 0.2
}

# Function to adjust otp15_takeoff and the top 5 correlated features excluding otp15_takeoff itself
def adjust_otp_and_top_5_correlations(row, eco_corr, buss_corr):
    # Select the appropriate correlation matrix
    corr_matrix = buss_corr if row['cabin_in_surveyed_flight'] == 'Business' else eco_corr

    # Identify the top 5 features based on absolute correlation with 'otp15_takeoff'
    # Exclude 'otp15_takeoff' from being considered as one of the top correlated features
    top_features = ['pfl_100_checkin_satisfaction', 'pfl_500_boarding_satisfaction', 'arr_100_arrivals_satisfaction', 'con_100_connections_satisfaction', 'ifl_100_cabin_crew_satisfaction']

    # Apply monthly improvement
    month = row['end_date'].month
    delta_otp15_takeoff = month_improvements.get(month, 0)
    row['otp15_takeoff'] += delta_otp15_takeoff
    
    # Apply correlation-based adjustments only to the top 5 correlated features
    for column in top_features:
        if column in row:
            adjustment_factor = corr_matrix.at[column, 'otp15_takeoff'] * delta_otp15_takeoff
            row[column] += adjustment_factor

    row['insert_date_ci'] = '2024-05-08'
    return row

# Adjust the DataFrame
adjusted_df = targets_df.apply(lambda row: adjust_otp_and_top_5_correlations(row.copy(), eco_corr, buss_corr), axis=1)
concatenated_df = pd.concat([targets_df, adjusted_df]).sort_index(kind='merge')





In [ ]:
concatenated_df

In [ ]:
concatenated_df=all_weekly.copy()
concatenated_df['insert_date_ci']='2024-06-01'

In [ ]:
def process_dataframe(df):
    df.drop(columns=['pun_100_punctuality_satisfaction', 'inm_400_issues_response_satisfaction'], inplace=True)
    # Agrupar y procesar los datos
    grouped_dfs = {}
    features = {}
    for group_name, group_data in df.groupby(['cabin_in_surveyed_flight', 'haul']):
        cabin_value, haul_value = group_name
        group_df = group_data.copy()
        group_df_name = f'{cabin_value}_{haul_value}_df'
        
        # Identificar las columnas de características
        satisfaction_cols = [col for col in df.columns if col.endswith('_satisfaction')]
        otp_cols = ['otp15_takeoff']
        features_cols = satisfaction_cols + ['load_factor'] + otp_cols
        cols_to_keep = ['insert_date_ci', 'start_date','end_date','cabin_in_surveyed_flight', 'haul'] + features_cols + ['NPS_weighted']

        # Filtrar las columnas en el grupo y actualizar el diccionario de características
        grouped_df = group_df[cols_to_keep]
        features[group_df_name] = features_cols
        grouped_dfs[group_df_name] = grouped_df

    # Reconstruir el DataFrame original
    df = pd.concat(grouped_dfs.values())
    df.reset_index(drop=True, inplace=True)

    return df, grouped_dfs, features

# Aplicar la función a cada DataFrame y almacenar los resultados en las variables correspondientes
day_predict_df, day_predict_df_grouped_dfs, features_cols = process_dataframe(concatenated_df)

In [ ]:
day_predict_df_grouped_dfs

In [ ]:
os.getcwd()

In [ ]:
satisfaction_cols = [col for col in day_predict_df.columns if col.endswith('_satisfaction')]
otp_cols = ['otp15_takeoff']
features_cols = satisfaction_cols + ['load_factor'] + otp_cols

In [ ]:
import pandas as pd
import numpy as np
import pickle
import os
import pandas as pd
from darts.timeseries import TimeSeries
import os
import pickle


def compute_shap_and_prediction(row, key, features_cols):
    """
    Computes SHAP values and the predicted NPS for a given row.
    
    Parameters:
    - row_df: The DataFrame row for which to compute SHAP values and prediction.
    - key: The key identifying the specific model and scaler to use.
    - features_cols: List of column names representing features used by the model.
    
    Returns:
    - A tuple containing SHAP values as a dictionary and the predicted NPS.
    """
    # Logic to prepare the row for SHAP value computation and prediction
    aux_nps_ts = TimeSeries.from_series(pd.Series([0]))
    aux_row = pd.DataFrame(0, index=[0], columns=row.columns)
    row_df = pd.concat([aux_row, row]).reset_index(drop=True)
    
    # Load the pre-trained model and scaler
    best_tuned_model_dataframe_path = os.path.join('targets_model', f"best_tuned_dataframe_{key}.pkl")
    with open(best_tuned_model_dataframe_path, 'rb') as dataframe_file:
        best_tuned_model = pickle.load(dataframe_file)
    
    future_scaler_path = os.path.join('targets_model', f"future_scaler_{key}.pkl")
    with open(future_scaler_path, 'rb') as scaler_file:
        future_scaler = pickle.load(scaler_file)
    
    future_covariates_ts = TimeSeries.from_dataframe(row_df[features_cols])[-1:]
    future_covariates_ts_scaled = future_scaler.transform(future_covariates_ts)
    
    model_file_path = os.path.join('targets_model', f"best_tuned_mae_model_{key}_{best_tuned_model['model_name']}.pkl")
    with open(model_file_path, 'rb') as model_file:
        model = pickle.load(model_file)
    
    # Compute SHAP values and prediction
    shap_explain = ShapExplainer(model=model)
    shap_explained = shap_explain.explain(aux_nps_ts, foreground_future_covariates=future_covariates_ts_scaled)
    shap_explanation = shap_explained.get_shap_explanation_object(horizon=1)

    shap_values = shap_explanation[0].values
    base_value = shap_explanation[0].base_values
    pred_value = base_value + shap_values.sum()
    feature_names=[]
    for feat in shap_explanation.feature_names:
        name = [f for f in features_cols if f in feat]
        feature_names.append(name[0])
    
    
    # Convert SHAP values to a dictionary and adjust the logic based on your ShapExplainer
    shap_values_dict = {f"{feature}_nps": value for feature, value in zip(feature_names, shap_values)}
    shap_values_dict["out_prob_base"] = base_value,
    shap_values_dict["out_prob_nps"] = pred_value,
    
    # print(row_df.loc[1,features_cols])
    
    shap_explanation = shap.Explanation(values=shap_values, 
                                 base_values=base_value, 
                                 data=np.array(row_df.loc[1,features_cols].values.flatten().tolist()), 
                                 feature_names=shap_explanation.feature_names)
    
    return shap_values_dict, shap_explanation, shap_explain, shap_explained


# Initialize a dictionary to store the augmented DataFrames
augmented_dfs = {}
explanations = {}

for key in day_predict_df_grouped_dfs.keys():
    # Initialize a list to collect augmented rows
    augmented_rows = []
    explanations[key]={}

    for index in range(len(day_predict_df_grouped_dfs[key])):
        # Access the row by its index using .iloc
        row_df = day_predict_df_grouped_dfs[key].iloc[[index]]

        # Compute SHAP values and predicted NPS here...
        # Assuming `compute_shap_and_prediction` is a function you'd implement
        # This function should return SHAP values as a dict and the predicted NPS
        shap_values, explanations[key][index], shap_explain, shap_explained = compute_shap_and_prediction(row_df, key, features_cols)
        # shap.plots.waterfall(explanations[key][index], max_display=20)

        # For each feature, add its SHAP value to the row
        for feature_name, shap_value in shap_values.items():
            row_df[f'{feature_name}'] = shap_value

        # Add base value and predicted NPS columns
        # row_df['Base Value'] = shap_values['base_value']  # Adjust based on how you obtain the base value
        # row_df['Predicted NPS'] = predicted_nps
        # print(key)
        # shap_explain.summary_plot()

        # Append the augmented row to the list
        augmented_rows.append(row_df)
        

    # Concatenate all augmented rows to form the complete augmented DataFrame
    augmented_dfs[key] = pd.concat(augmented_rows).reset_index(drop=True)

# `augmented_dfs` now contains the augmented DataFrames with SHAP values and predictions
augmented_dfs

In [ ]:
model_file_path = os.path.join('targets_model', f"best_tuned_mae_model_Economy_SH_df_LightGBMModel.pkl")
with open(model_file_path, 'rb') as model_file:
    model = pickle.load(model_file)

In [ ]:
model

In [ ]:
# Access the trained LightGBM model
lgb_model = model.model
feature_names = model.lagged_feature_names

# Set feature names manually if not already set
lgb_model.feature_name_ 

In [ ]:
feature_names

In [ ]:

lightgbm.plot_importance(lgb_model, importance_type="gain", figsize=(7,6), title="LightGBM Feature Importance (Gain)")
plt.show()

In [ ]:
shap_explain = ShapExplainer(model=model)
shap_explain.summary_plot()

In [ ]:
augmented_dfs

In [ ]:
bsh_xlsx = augmented_dfs['Business_SH_df'][['cabin_in_surveyed_flight', 'haul', 'insert_date_ci', 'start_date', 'end_date', 'otp15_takeoff', 'out_prob_nps']]

In [ ]:
bsh_xlsx.to_excel('business_sh_inc_otp_08052024.xlsx')

In [ ]:
eco_xlsx = augmented_dfs['Economy_SH_df'][['cabin_in_surveyed_flight', 'haul', 'insert_date_ci', 'start_date', 'end_date', 'otp15_takeoff', 'out_prob_nps']]

In [ ]:
eco_xlsx.to_excel('economy_sh_inc_otp_08052024.xlsx')

In [ ]:
chinese_LH

In [ ]:
augmented_dfs['Business_LH_df']

In [ ]:
augmented_dfs['Business_SH_df']

In [ ]:
augmented_dfs['Economy_LH_df']

In [ ]:
augmented_dfs['Business_LH_df']

In [ ]:
overall_LH_explanation=explanations['Economy_LH_df'][0]

In [ ]:
explanations['Business_LH_df']

In [ ]:
chinese_LH_explanation=explanations['Economy_LH_df'][1]

In [ ]:
def create_uplifting_explanation(explanation2, explanation1):
    """
    Create a new Explanation object representing the uplifting between two Explanation objects.

    Parameters:
        - explanation1: The first shap.Explanation object.
        - explanation2: The second shap.Explanation object.

    Returns:
        - A new shap.Explanation object representing the uplifting.
    """
    # Calculate the difference in values, base_values, and data
    diff_values = explanation2.values - explanation1.values
    
    diff_base_values = explanation1.base_values + sum(explanation1.values)
    diff_data = explanation2.data - explanation1.data

    # Create a new Explanation object with the difference values
    diff_explanation = shap.Explanation(values=diff_values, base_values=diff_base_values, data=diff_data,
                                        feature_names=explanation1.feature_names)

    return diff_explanation

In [ ]:
march_diff_explanation = create_uplifting_explanation(chinese_LH_explanation, overall_LH_explanation)

In [ ]:
shap.plots.waterfall(overall_LH_explanation, max_display=30)

In [ ]:
augmented_dfs['Business_LH_df'].to_excel('shaps_for_march_and_april_Business_LH_comparison.xlsx')

In [ ]:
    # Reconstruir el DataFrame original
df = pd.concat(augmented_dfs.values())
df.reset_index(drop=True, inplace=True)

In [ ]:
df.to_csv('weekly_predictions.csv')

In [ ]:
df = pd.read_csv('weekly_predictions.csv')

In [ ]:
df.drop(columns=['Unnamed: 0'])

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

# Suponemos que df es tu DataFrame y ya está definido y cargado con los datos necesarios.
haul = 'SH'
cabin = 'Economy'
variable = 'otp15_takeoff'
real_target = 'NPS_weighted'
predicted_target = 'out_prob_nps'

# Define la función para remover outliers
def remove_outliers(data, threshold=1.5):
    q25 = np.percentile(data, 25)
    q75 = np.percentile(data, 75)
    iqr = q75 - q25
    lower_bound = q25 - threshold * iqr
    upper_bound = q75 + threshold * iqr
    return (data >= lower_bound) & (data <= upper_bound)

# Filtrado de datos
filtered_data = df[(df['haul'] == haul) & (df['cabin_in_surveyed_flight'] == cabin)]
x = filtered_data[variable].to_numpy().reshape(-1, 1)  # Características
y_real = filtered_data[real_target].to_numpy()  # Valores reales
y_pred = filtered_data[predicted_target].to_numpy()  # Valores predichos

# Identificar índices de datos válidos sin outliers para x y y_real
valid_x = remove_outliers(x.flatten())
valid_y_real = remove_outliers(y_real)

# Filtrar x, y_real, y_pred usando índices válidos
valid_indices = valid_x & valid_y_real
x_clean = x[valid_indices].reshape(-1, 1)
y_real_clean = y_real[valid_indices]
y_pred_clean = y_pred[valid_indices]

# Ajustar el modelo de regresión lineal con datos limpios
model = LinearRegression()
model.fit(x_clean, y_real_clean)

# Generar valores para la línea de regresión
x_fit = np.linspace(x_clean.min(), x_clean.max(), 100).reshape(-1, 1)
y_fit = model.predict(x_fit)

# Crear el gráfico de dispersión con la línea de regresión
plt.figure(figsize=(10,6))
plt.scatter(x_clean, y_real_clean, color='black', alpha=0.6, edgecolors='w', linewidths=0.5, label='Real NPS')
plt.title(f'Impact of {variable} on NPS - Real vs. Predicted with Regression')
plt.xlabel(f'{variable} Values')
plt.ylabel('NPS Values')
plt.grid(True)
plt.legend()



In [ ]:
# Crear el gráfico de dispersión con la línea de regresión
plt.figure(figsize=(10,6))
plt.scatter(x_clean, y_real_clean, color='blue', alpha=0.6, edgecolors='w', linewidths=0.5, label='Real NPS')
plt.plot(x_fit, y_fit, color='green', linewidth=2, label='Regression Line')  # Añadir la línea de regresión
plt.title(f'Impact of {variable} on NPS - Real vs. Predicted with Regression')
plt.xlabel(f'{variable} Values')
plt.ylabel('NPS Values')
plt.grid(True)
plt.legend()

# Mover la anotación a la esquina inferior izquierda
plt.annotate(f'Slope: {model.coef_[0]:.2f}\nIntercept: {model.intercept_:.2f}\nR² Score: {model.score(x_clean, y_real_clean):.2f}', 
             xy=(0.05, 0.05), xycoords='axes fraction', verticalalignment='bottom', 
             bbox=dict(boxstyle="round,pad=0.3", edgecolor='green', facecolor='white'))
plt.show()

In [ ]:
# Crear el gráfico de dispersión con la línea de regresión
plt.figure(figsize=(10,6))
plt.scatter(x_clean, y_real_clean, color='blue', alpha=0.6, edgecolors='w', linewidths=0.5, label='Real NPS')
plt.scatter(x_clean, y_pred_clean, color='red', alpha=0.6, edgecolors='w', linewidths=0.5, label='Predicted NPS')
plt.plot(x_fit, y_fit, color='green', linewidth=2, label='Regression Line')  # Añadir la línea de regresión
plt.title(f'Impact of {variable} on NPS - Real vs. Predicted with Regression')
plt.xlabel(f'{variable} Values')
plt.ylabel('NPS Values')
plt.grid(True)
plt.legend()

# Mover la anotación a la esquina inferior izquierda
plt.annotate(f'Slope: {model.coef_[0]:.2f}\nIntercept: {model.intercept_:.2f}\nR² Score: {model.score(x_clean, y_real_clean):.2f}', 
             xy=(0.05, 0.05), xycoords='axes fraction', verticalalignment='bottom', 
             bbox=dict(boxstyle="round,pad=0.3", edgecolor='green', facecolor='white'))
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np

# variable = 'ifl_100_cabin_crew_satisfaction'
variable = 'otp15_takeoff'
target = f'{variable}_nps'
# target = 'out_prob_nps'
# target= 'NPS_weighted'


# Filter the data
filtered_data = df[(df['haul'] == haul) & (df['cabin_in_surveyed_flight'] == cabin)]
x = filtered_data[variable].to_numpy().reshape(-1, 1)  # Features
y = filtered_data[target].to_numpy()  # Target

# Define a function to remove outliers
def remove_outliers(data, threshold=1.5):
    q25 = np.percentile(data, 10)
    q75 = np.percentile(data, 90)
    iqr = q75 - q25
    lower_bound = q25 - threshold * iqr
    upper_bound = q75 + threshold * iqr
    return (data >= lower_bound) & (data <= upper_bound)


# Assuming 'df' is your DataFrame and x, y are already defined as numpy arrays.
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

# Filter data to get the relevant subset
filtered_data = df[(df['haul'] == haul) & (df['cabin_in_surveyed_flight'] == cabin)]
x = filtered_data[variable].values.reshape(-1, 1)  # Features as numpy array
y = filtered_data[target].values            # Target as numpy array

# Identify non-outlier indices for both x and y
valid_x = remove_outliers(x.flatten())  # flatten x to 1D for consistency with y
valid_y = remove_outliers(y)

# Get common indices where both x and y are non-outliers
valid_indices = valid_x & valid_y

# Filter both x and y using the valid_indices
x_clean = x[valid_indices].reshape(-1, 1)
y_clean = y[valid_indices]

# Fit the linear regression model with cleaned data
model_clean = LinearRegression()
model_clean.fit(x_clean, y_clean)

# Get model parameters
slope_clean = model_clean.coef_[0]
intercept_clean = model_clean.intercept_
r2_score_clean = model_clean.score(x_clean, y_clean)

# Generate values for the regression line
x_fit = np.linspace(x_clean.min(), x_clean.max(), 100).reshape(-1, 1)
y_fit = model_clean.predict(x_fit)

# Create the scatter plot with regression line
plt.figure(figsize=(10, 6))
plt.scatter(x_clean, y_clean, alpha=0.6, edgecolors='w', linewidths=0.5)
plt.plot(x_fit, y_fit, color='red', linewidth=2)  # Add the regression line
plt.title(f'{variable} vs {target}')
plt.xlabel(f'{variable} Actual Values')
plt.ylabel(f'{target}')
plt.grid(True)
plt.annotate(f'Slope: {slope_clean:.2f}\nIntercept: {intercept_clean:.2f}\nR² Score: {r2_score_clean:.2f}', xy=(0.05, 0.95), xycoords='axes fraction', 
             verticalalignment='top', bbox=dict(boxstyle="round,pad=0.3", edgecolor='red', facecolor='white'))
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor

# Generate synthetic data with a non-linear relationship
np.random.seed(0)
X1 = np.random.uniform(-3, 3, 100)
X2 = np.random.uniform(-3, 3, 100)
Y = np.sin(X1) + np.cos(X2) + np.random.normal(0, 0.1, 100)  # Non-linear relationship

# Fit a non-linear model
model = RandomForestRegressor()
model.fit(np.column_stack((X1, X2)), Y)

# Predictions for visualization
x1_range = np.linspace(-3, 3, 100)
x2_range = np.linspace(-3, 3, 100)
X1_grid, X2_grid = np.meshgrid(x1_range, x2_range)
Y_pred = model.predict(np.c_[X1_grid.ravel(), X2_grid.ravel()]).reshape(X1_grid.shape)

# Plot
plt.figure(figsize=(10, 7))
plt.contourf(X1_grid, X2_grid, Y_pred, levels=30, cmap='viridis')
plt.colorbar()
plt.xlabel('X1')
plt.ylabel('X2')
plt.title('Predicted Y from Non-Linear Model')
plt.show()





In [ ]:
targets_pred = df[(df['insert_date_ci']=='2023-11-21')&(pd.to_datetime(df['end_date']).dt.year>2023)]

In [ ]:
targets_pred[['start_date', 'end_date','cabin_in_surveyed_flight', 'haul', 'NPS_weighted', 'out_prob_nps', 'out_prob_base']]
targets_pred

In [ ]:
df= targets_pred.copy()
# Calcular la diferencia entre NPS_weighted y out_prob_nps
df['difference'] = df['NPS_weighted'] - df['out_prob_nps']

# Obtener las columnas de SHAP que terminan en "_nps"
shap_columns = [col for col in df.columns if col.endswith('_nps') and col != 'out_prob_nps']

# Función para ajustar los valores de SHAP
def adjust_shap_values(row):
    difference = row['difference']
    if difference > 0:
        # Filtrar los valores SHAP positivos
        positive_shaps = [col for col in shap_columns if row[col] > 0]
        if positive_shaps:
            adjustment = difference / len(positive_shaps)
            for col in positive_shaps:
                row[col] += adjustment
    elif difference < 0:
        # Filtrar los valores SHAP negativos
        negative_shaps = [col for col in shap_columns if row[col] < 0]
        if negative_shaps:
            adjustment = difference / len(negative_shaps)
            for col in negative_shaps:
                row[col] += adjustment
    return row

# Aplicar la función de ajuste a cada fila del dataframe
df = df.apply(adjust_shap_values, axis=1)

# Actualizar la columna out_prob_nps con la suma de los nuevos valores SHAP y el valor base
df['out_prob_nps'] = df[shap_columns].sum(axis=1) + df['out_prob_base']

# Eliminar la columna de diferencia ya que no es necesaria
df.drop(columns=['difference'], inplace=True)
df

In [ ]:
import numpy as np

def prepare_and_adjust_shap(df, shap_columns, desired_nps, base_output_prob, clamp_min, clamp_max):
    # Calculate the target SHAP sum, which is the desired NPS minus the base model output probability
    df['target_shap_sum'] = desired_nps - base_output_prob

    # Proceed to normalize and clamp SHAP values
    df = normalize_clamp_shap(df, shap_columns, df['target_shap_sum'], clamp_min, clamp_max)

    # Update 'out_prob_nps' with the new SHAP values sum and the base output probability
    df['out_prob_nps'] = df['out_prob_base'] + df[shap_columns].sum(axis=1)

    # Check if the new 'out_prob_nps' matches 'NPS_weighted'
    df['is_correct_nps'] = np.isclose(df['out_prob_nps'], df['NPS_weighted'], atol=1e-5)

    return df

def normalize_clamp_shap(df, shap_columns, target_shap_sum, clamp_min, clamp_max):
    # Calculate the initial sum of SHAP values
    current_shap_total = df[shap_columns].sum(axis=1)

    # Normalize SHAP values to match the target prediction error
    normalized_shap = df[shap_columns].div(current_shap_total, axis=0).mul(target_shap_sum, axis=0)

    # Apply proportional scaling to ensure all values are within bounds
    scaling_factor = np.maximum(np.abs(normalized_shap / clamp_max), np.abs(normalized_shap / clamp_min))
    scaling_factor = scaling_factor.max(axis=1)

    # Adjust scaling factor to avoid division by zero and ensure it's at least 1
    scaling_factor[scaling_factor < 1] = 1

    # Apply scaling
    adjusted_shap = normalized_shap.div(scaling_factor, axis=0)

    # Assign adjusted SHAP values back, ensuring they stay within bounds
    adjusted_shap = adjusted_shap.clip(lower=clamp_min, upper=clamp_max)
    df.loc[:, shap_columns] = adjusted_shap

    return df

# Example usage:
clamp_min, clamp_max = -5, 5  # Define bounds
desired_nps = targets_pred['NPS_weighted']  # Assuming this is your desired NPS
base_output_prob = targets_pred['out_prob_base']  # Assuming this is your base output probability

adjusted_df = prepare_and_adjust_shap(targets_pred.copy(), shap_columns, desired_nps, base_output_prob, clamp_min, clamp_max)
print(adjusted_df[['out_prob_nps', 'NPS_weighted', 'is_correct_nps']])




In [ ]:
adjusted_df

In [ ]:
import numpy as np

def prepare_and_adjust_shap(df, shap_columns, desired_nps, base_output_prob, clamp_min, clamp_max):
    # Calculate the target SHAP sum, which is the desired NPS minus the base model output probability
    df['target_shap_sum'] = desired_nps - base_output_prob

    # Normalize and clamp SHAP values
    df = normalize_clamp_shap(df, shap_columns, df['target_shap_sum'], clamp_min, clamp_max)

    # Update 'out_prob_nps' with the new SHAP values sum and the base output probability
    df['out_prob_nps'] = df['out_prob_base'] + df[shap_columns].sum(axis=1)

    # Apply a final normalization if 'out_prob_nps' does not match 'NPS_weighted'
    df = final_normalization(df, shap_columns, desired_nps)

    # Check if the new 'out_prob_nps' matches 'NPS_weighted'
    df['is_correct_nps'] = np.isclose(df['out_prob_nps'], df['NPS_weighted'], atol=1e-5)

    return df

def normalize_clamp_shap(df, shap_columns, target_shap_sum, clamp_min, clamp_max):
    current_shap_total = df[shap_columns].sum(axis=1)
    normalized_shap = df[shap_columns].div(current_shap_total, axis=0).mul(target_shap_sum, axis=0)
    clamped_shap = normalized_shap.clip(lower=clamp_min, upper=clamp_max)
    df[shap_columns] = clamped_shap
    return df

def final_normalization(df, shap_columns, desired_nps):
    # Calculate the total contribution needed from SHAP values to meet the desired NPS
    total_needed_shap = desired_nps - df['out_prob_base']
    current_shap_sum = df[shap_columns].sum(axis=1)

    # Determine the factor by which to adjust the SHAP values
    normalization_factor = total_needed_shap / current_shap_sum

    # Adjust SHAP values
    df[shap_columns] = df[shap_columns].mul(normalization_factor, axis=0)
    df['out_prob_nps'] = df['out_prob_base'] + df[shap_columns].sum(axis=1)
    return df

# Example usage:
clamp_min, clamp_max = -5, 5  # Define bounds
desired_nps = targets_pred['NPS_weighted']  # Assuming this is your desired NPS
base_output_prob = targets_pred['out_prob_base']  # Assuming this is your base output probability

adjusted_df = prepare_and_adjust_shap(targets_pred.copy(), shap_columns, desired_nps, base_output_prob, clamp_min, clamp_max)
print(adjusted_df[['out_prob_nps', 'NPS_weighted', 'is_correct_nps']])


In [ ]:
# Filter the DataFrame based on the conditions
condition = (
    (adjusted_df['cabin_in_surveyed_flight'] == 'Business') & 
    (adjusted_df['haul'] == 'LH') & 
    (adjusted_df['start_date'] == '2024-01-01')
)

# Set the 'loy_200_loyalty_programme_satisfaction_nps' column value to -3.6 for these rows
adjusted_df.loc[condition, 'loy_200_loyalty_programme_satisfaction_nps'] = -3.6


In [ ]:
# Filter the DataFrame based on the conditions
condition = (
    (adjusted_df['cabin_in_surveyed_flight'] == 'Business') & 
    (adjusted_df['haul'] == 'LH') & 
    (adjusted_df['start_date'] == '2024-01-01')
)

# Set the 'loy_200_loyalty_programme_satisfaction_nps' column value to -3.6 for these rows
adjusted_df.loc[condition, 'img_310_ease_contact_phone_satisfaction_nps'] = -1.2

In [ ]:
# Filter the DataFrame based on the conditions
condition = (
    (adjusted_df['cabin_in_surveyed_flight'] == 'Premium Economy') & 
    (adjusted_df['haul'] == 'LH') & 
    (adjusted_df['start_date'] == '2024-08-01')
)

# Set the 'loy_200_loyalty_programme_satisfaction_nps' column value to -3.6 for these rows
adjusted_df.loc[condition, 'ifl_100_cabin_crew_satisfaction_nps'] = 6


In [ ]:
# Filter the DataFrame based on the conditions
condition = (
    (adjusted_df['cabin_in_surveyed_flight'] == 'Premium Economy') & 
    (adjusted_df['haul'] == 'LH') & 
    (adjusted_df['start_date'] == '2024-08-01')
)

# Set the 'loy_200_loyalty_programme_satisfaction_nps' column value to -3.6 for these rows
adjusted_df.loc[condition, 'ifl_400_food_drink_satisfaction_nps'] = 4.332

In [ ]:
# Filter the DataFrame based on the conditions
condition = (
    (adjusted_df['cabin_in_surveyed_flight'] == 'Premium Economy') & 
    (adjusted_df['haul'] == 'LH') & 
    (adjusted_df['start_date'] == '2024-08-01')
)

# Set the 'loy_200_loyalty_programme_satisfaction_nps' column value to -3.6 for these rows
adjusted_df.loc[condition, 'pfl_100_checkin_satisfaction_nps'] = -1.5

In [ ]:
# Filter the DataFrame based on the conditions
condition = (
    (adjusted_df['cabin_in_surveyed_flight'] == 'Premium Economy') & 
    (adjusted_df['haul'] == 'LH') & 
    (adjusted_df['start_date'] == '2024-10-01')
)

# Set the 'loy_200_loyalty_programme_satisfaction_nps' column value to -3.6 for these rows
adjusted_df.loc[condition, 'otp15_takeoff_nps'] = 4.014

In [ ]:
# Filter the DataFrame based on the conditions
condition = (
    (adjusted_df['cabin_in_surveyed_flight'] == 'Premium Economy') & 
    (adjusted_df['haul'] == 'LH') & 
    (adjusted_df['start_date'] == '2024-10-01')
)

# Set the 'loy_200_loyalty_programme_satisfaction_nps' column value to -3.6 for these rows
adjusted_df.loc[condition, 'load_factor_nps'] = 1.863

In [ ]:
# Filter the DataFrame based on the conditions
condition = (
    (adjusted_df['cabin_in_surveyed_flight'] == 'Premium Economy') & 
    (adjusted_df['haul'] == 'LH') & 
    (adjusted_df['start_date'] == '2024-10-01')
)

# Set the 'loy_200_loyalty_programme_satisfaction_nps' column value to -3.6 for these rows
adjusted_df.loc[condition, 'img_310_ease_contact_phone_satisfaction_nps'] = 2.085

In [ ]:
# Filter the DataFrame based on the conditions
condition = (
    (adjusted_df['cabin_in_surveyed_flight'] == 'Premium Economy') & 
    (adjusted_df['haul'] == 'LH') & 
    (adjusted_df['start_date'] == '2024-10-01')
)

# Set the 'loy_200_loyalty_programme_satisfaction_nps' column value to -3.6 for these rows
adjusted_df.loc[condition, 'ifl_400_food_drink_satisfaction_nps'] = -4.5

In [ ]:
# Filter the DataFrame based on the conditions
condition = (
    (adjusted_df['cabin_in_surveyed_flight'] == 'Premium Economy') & 
    (adjusted_df['haul'] == 'LH') & 
    (adjusted_df['start_date'] == '2024-10-01')
)

# Set the 'loy_200_loyalty_programme_satisfaction_nps' column value to -3.6 for these rows
adjusted_df.loc[condition, 'pfl_100_checkin_satisfaction_nps'] = 4.092

In [ ]:
# Filter the DataFrame based on the conditions
condition = (
    (adjusted_df['cabin_in_surveyed_flight'] == 'Premium Economy') & 
    (adjusted_df['haul'] == 'LH') & 
    (adjusted_df['start_date'] == '2024-10-01')
)

# Set the 'loy_200_loyalty_programme_satisfaction_nps' column value to -3.6 for these rows
adjusted_df.loc[condition, 'ifl_100_cabin_crew_satisfaction_nps'] = -2.592

In [ ]:
# Filter the DataFrame based on the conditions
condition = (
    (adjusted_df['cabin_in_surveyed_flight'] == 'Premium Economy') & 
    (adjusted_df['haul'] == 'LH') & 
    (adjusted_df['start_date'] == '2024-10-01')
)

# Set the 'loy_200_loyalty_programme_satisfaction_nps' column value to -3.6 for these rows
adjusted_df.loc[condition, 'ifl_200_flight_crew_annoucements_satisfaction_nps'] = -0.254

In [ ]:
def check_final(df, shap_columns, desired_nps, base_output_prob):
    # Update 'out_prob_nps' with the new SHAP values sum and the base output probability
    df['out_prob_nps'] = df['out_prob_base'] + df[shap_columns].sum(axis=1)

    # Check if the new 'out_prob_nps' matches 'NPS_weighted'
    df['is_correct_nps'] = np.isclose(df['out_prob_nps'], df['NPS_weighted'], atol=1e-5)

    return df
adjusted_df = check_final(adjusted_df.copy(), shap_columns, desired_nps, base_output_prob)

In [ ]:
print(adjusted_df[['out_prob_nps', 'NPS_weighted', 'is_correct_nps']])

In [ ]:
df = final_normalization(adjusted_df, shap_columns, desired_nps)

In [ ]:
np.isclose(df['out_prob_nps'], df['NPS_weighted'], atol=1e-5)

In [ ]:
adjusted_df.tail()

In [ ]:
def add_shap_sum_column(df, shap_columns):
    # Calculate the sum of SHAP values across the specified columns for each row
    df['sum_adjusted_shaps'] = df[shap_columns].sum(axis=1) + df['out_prob_base']
    return df

# Apply the function to add the sum column to the adjusted DataFrame
adjusted_df = add_shap_sum_column(adjusted_df, shap_columns)

In [ ]:
adjusted_df[['start_date', 'end_date','cabin_in_surveyed_flight', 'haul', 'NPS_weighted', 'out_prob_nps','sum_adjusted_shaps', 'out_prob_base']]

In [ ]:
# Step 1: Identify all Shapley columns, excluding 'out_prob_nps'
shap_columns = [col for col in targets_pred.columns if col.endswith('_nps') and col != 'out_prob_nps']
print(shap_columns)

# Step 2: Calculate the current total Shapley values per row
current_shap_total = targets_pred[shap_columns].sum(axis=1)

# Step 3: Calculate the target Shapley sum (NPS_weighted - out_prob_base)
target_shap_sum = targets_pred['NPS_weighted'] - targets_pred['out_prob_base']

# Step 4: Calculate the adjustment ratio
adjustment_ratio = target_shap_sum / current_shap_total

# Step 5: Adjust Shapley values using the adjustment ratio
for col in shap_columns:
    targets_pred[col] *= adjustment_ratio



In [ ]:
df

In [ ]:
df.to_excel('final_shaps_for_targets.xlsx')

# Debug concatenated targets

In [ ]:
targets_df= pd.read_csv('corrected_nps_data.csv')

In [ ]:
df_historic = pd.read_csv('historic_predictions_q1 (1).csv')

In [ ]:
concatenated_df = pd.concat([df_historic,targets_df.drop(columns='NPS_weighted')], ignore_index=True)

In [ ]:
concatenated_df